In [225]:
#imports 
import multiprocessing
import datetime
import feather
import matplotlib as mpt
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy as sc
import seaborn as sns
import sys
import pickle
import os
import shutil
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

In [226]:
today = datetime.date.today().isoformat()
df_aeolus = feather.read_dataframe('Data/AEOLUS_clean_ranked.feather')
drug = 'Aspirin'
drugID = df_aeolus.query('drug_concept_name==@drug').get('drug_concept_id').values.item(0)


In [253]:
# workflow functions:
def match(df, cal=0.05, minCtrl = 10, numBins = 100):
    allCases = df.copy(deep=True).query('{}==1'.format(drug))
    allControls = df.query('{}==0'.format(drug))
    
    if (len(allCases)<50):
        print('insufficient cases')
        return
    
    allCases['bin'] = pd.cut(allCases.Propensity, numBins)
    mCases, mControls = [], []
    
    for cBin, caseData in allCases.groupby(by='bin'):
        minPS, maxPS = cBin.left, cBin.right
        numCtrl = len(caseData)*minCtrl
        
        controlOptions = allControls.query('Propensity > (@minPS-@cal) and Propensity < (@maxPS+@cal)').get('ID').values
        
        if (len(controlOptions)>minCtrl):
            mCases = np.append(mCases, caseData.get('ID').values)
            mControls = np.append(mControls, (np.random.choice(controlOptions, numCtrl)))
    
    return mCases, mControls

def selectData(cases, controls, source=df_aeolus, drug=drug):
    selection = []

    selection.append(source[source.id.isin(cases)])

    ctrl_unique, ctrl_counts = np.unique(controls, return_counts=True)

    for count in np.unique(ctrl_counts):  
        idx = np.where(ctrl_counts==count)
        ctrls = np.take(ctrl_unique, idx)

        toAdd = source[source.id.isin(np.hstack(ctrls))]
        for i in range(0,count): 
                selection.append(toAdd)

    return pd.concat(selection, ignore_index=True)

def doChiSquareX(cases, source=df_matched, drugName=drug, drugID=drugID):
    chiResult = []
    df_drug = source[source.id.isin(cases)].query('drug_concept_id==@drugID')

    XM = df_drug.query('gender_code=="M"').shape[0]
    XF = df_drug.query('gender_code=="F"').shape[0]

    for outID in df_drug.get('snomed_outcome_concept_id').unique():

        df_out = df_drug.query('snomed_outcome_concept_id==@outID')

        outName = df_out.iloc[0].get('outcome_concept_name')
        outRank = df_out.iloc[0].get('outcome_rank')

        XME = df_out.query('gender_code=="M"').shape[0]
        XFE = df_out.query('gender_code=="F"').shape[0]

        if ((not XME == 0) and (not XFE == 0)):
            contingencyTable = [[XFE, XF], [XME, XM]]
        else:
            contingencyTable = [[1, 1], [1, 1]]

        chi2, p, dof, expected = sc.stats.chi2_contingency(contingencyTable)

        cols = ['snomed_outcome_concept_id', 'outcome_concept_name', 'outcome_rank', 'p_valueX', 'XFE', 'XME', 'XM', 'XF']
        data = [[outID, outName, outRank, float(p), XFE, XME, XM, XF]]

        chiResult.append(pd.DataFrame(columns=cols, data=data))

    return pd.concat(chiResult, ignore_index=True) 

def bonferroni(df, colName):
    p_values = pd.to_numeric(df[colName])
    bonferroni_pvalues = p_values * len(p_values)
    df = df.assign(newCol=bonferroni_pvalues)
    newCol = 'bonf_' + colName
    return df.rename(columns={'newCol': newCol})

def getPairsX(cases, alpha=0.05):

    df_chi = doChiSquareX(cases)
    df_bonf = bonferroni(df=df_chi, colName='p_valueX' )

    return df_bonf.query('bonf_p_valueX<=@alpha')

def doChiSquareE(df, source=df_matched, drugName=drug):
    
    TF = source.query('gender_code=="F"').shape[0]
    TM = source.query('gender_code=="M"').shape[0]
    
    for idx, data in df.iterrows():
        
        df_out = source.query('snomed_outcome_concept_id==@data.snomed_outcome_concept_id')
        
        TFE = df_out.query('gender_code=="F"').shape[0]
        TME = df_out.query('gender_code=="M"').shape[0]
        
        contingencyTable = [[TFE, TF], [TME, TM]]
        chi2, p, dof, expected = sc.stats.chi2_contingency(contingencyTable)
        
        df.at[idx, 'p_valueE'] = p
        df.at[idx, 'TFE'] = TFE
        df.at[idx, 'TME'] = TME 
        
    return df

def getE(df=df_pairsX, alpha=0.05):
    df_chi = doChiSquareE(df)
    df_bonf = bonferroni(df=df_chi, colName='p_valueE' )
    
    df_bonf['p_valueE'] = df_bonf.get('p_valueE').mask(df_bonf.get('p_valueE')>alpha, float('nan'))
    
    return df_bonf

def RORe(df, source=df_matched, drug=drug):
    
    TF = source.query('gender_code=="F"').shape[0]
    TM = source.query('gender_code=="M"').shape[0]
    
    df['TFe'] = TF-df['TFE']
    df['TMe'] = TM-df['TME']
    
    df.eval('RORE = log((TFE/TFe)/(TME/TMe))', inplace=True)
    
    return df

def RORx(df): 
    df['XFe'] = df['XF']-df['XFE']
    df['XMe'] = df['XM']-df['XME']
    
    df.eval('RORX = log((XFE/XFe)/(XME/XMe))', inplace=True)
    
    return df

def normalize(df, cols=['RORX','RORE']):
    for col in cols: 
        x = np.fabs(df[col])
        x = (x-x.min())/(x.max()-x.min())
        df = df.assign(xNorm=x).rename(columns={'xNorm': col+"N"})
        
    return df

def getRisk(df):
    
    for i, data in df.iterrows():
        e = data.RORE
        x = data.RORX

        if(data.p_valueE):
            if(x<0):
                if((e<0)&(x<e)):
                    df.at[i, 'RORa'] = x-e
            else:
                if((e>0)&(e<x)):
                    df.at[i, 'RORa'] = x-e
                
    df_risk = df
    df_risk.query('RORX>1 | RORX<-1', inplace=True)
    df_risk = normalize(df_risk, cols=['RORa']).eval('risk=RORaN*outcome_rank')
    
    for i, data in df_risk.iterrows():
        if(data.RORa<0):
            df_risk.at[i, 'risk'] = data.risk*(-1)
    
    return df_risk

In [228]:
scores = feather.read_dataframe("{}/PSM/LRCV pscores.feather".format(drug))

cases, controls = match(scores)

df_matched = selectData(cases, controls)

In [230]:
df_pairsX = getPairsX(cases)

In [250]:
df_XE = getE(df_pairsX)

In [234]:
df_ROR = RORe(RORx(df_XE))

In [236]:
df_risk = getRisk(df_ROR)

In [294]:
f = "Highest Risks to Females: \n"+df_risk.nlargest(5, 'risk')\
        .get(['outcome_concept_name','risk','snomed_outcome_concept_id'])\
        .set_index(np.arange(1,6))\
        .query('risk>0')\
        .to_string(header=False)
    
print(s)

Highest Risks to Females: 
1  Disorder.Of.Nail  0.009127  132706


In [297]:
m = "Highest Risks to Male: \n"+df_risk.nsmallest(5, 'risk')\
        .get(['outcome_concept_name','risk','snomed_outcome_concept_id'])\
        .set_index(np.arange(1,6))\
        .query('risk<0')\
        .to_string(header=False, justify='justify')
    
print(m)

Highest Risks to Male: 
1                  Viral.Hepatitis.C -0.263298  197494
2                    Dermatomyositis -0.243833   80182
3          Urinary.Tract.Obstruction -0.171459  194406
4          Ischemic.Optic.Neuropathy -0.145658  373487
5  Delay.When.Starting.To.Pass.Urine -0.139755  201688
